### Connect go Google Drive

In [ ]:
# mount google drive to access cloned repo
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Install Dependencies

In [ ]:
!pip install datasets
!pip install -U transformers
!pip install bitsandbytes==0.43.2
!pip install trl==0.14
!pip install evaluate
!pip install rouge_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 442.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 46.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

# Optimised Code

In [ ]:
cd drive/MyDrive/GitHub/finetuning-llm/

/content/drive/MyDrive/GitHub/finetuning-llm


In [ ]:
pwd

'/content/drive/MyDrive/GitHub/finetuning-llm'

In [ ]:
# Provide Colab a systemic path:
import sys
sys.path.append('/content/drive/My Drive/GitHub/finetuning-llm/')

In [ ]:
import torch
import os

from src.train.callbacks import BatchSizeCallback, MetricsLoggingCallback
from src.train.finetune_helpers import ModelArguments, ScriptArguments
from src.train.finetune_seq2seq import *

os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # log all model checkpoints

In [ ]:
model_args = ModelArguments(
    model_name="google-t5/t5-small",
    use_4bit=True,
    use_nested_quant=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_4bit_quant_dtype="nf4",
    lora_alpha=128,
    lora_dropout=0.1,
    lora_r=8
)

script_args = ScriptArguments(
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    auto_find_batch_size=True,
    # gradient_accumulation_steps=4,
    # eval_accumulation_steps=2,
    weight_decay=0.01,
    max_seq_length=512,
    dataset_name="4DR1455/finance_questions",
    bf16=True,
    learning_rate=2e-5,
    optim='adafactor',
    lr_scheduler_type='cosine',
    packing=True,
    num_train_epochs=9,
    # max_steps=20,
    save_steps=50,
    logging_steps=50,
    eval_steps=50,
    warmup_steps=50,
    eval_strategy='steps',
    run_name="Google-T5-Small-v3.0",
    report_to="wandb",
    save_safetensors=True,
    label_names=['labels'],
    load_best_model_at_end=True,
    dataloader_num_workers=10,
)

seq2seq_llm = FinetuneSeq2SeqLLMs(model_args=model_args, script_args=script_args)

tokenizer = seq2seq_llm.get_tokenizer()
peft_config = seq2seq_llm.get_peft_config()
peft_model = seq2seq_llm.get_model(peft_config)

data_collator = data_collator(tokenizer=tokenizer, peft_model=peft_model)

train_set, eval_set, test_set = seq2seq_llm.tokenize_split_dataset(sample=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

[09:49:29] None                                                                             finetune_seq2seq.py:198

README.md:   0%|          | 0.00/163 [00:00<?, ?B/s]

finance_questions_dataset.json:   0%|          | 0.00/53.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/53937 [00:00<?, ? examples/s]

Map:   0%|          | 0/17979 [00:00<?, ? examples/s]

In [ ]:
seq2seq_llm.train(
    peft_model=peft_model,
    peft_config=peft_config,
    data_collator=data_collator)

/content/drive/My Drive/GitHub/finetuning-llm/src/train/finetune_seq2seq.py:318: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

wandb: Adding directory to artifact (./results/google-t5/t5-small/checkpoint-400)... Done. 0.0s


[10:38:32] Evaluation metrics: {'eval_loss': 0.13895495235919952, 'eval_R1': 0.2899, 'eval_R2':     callbacks.py:39
           0.0664, 'eval_RL': 0.2781, 'eval_RLsum': 0.2778, 'eval_runtime': 7.817,                                 
           'eval_samples_per_second': 230.395, 'eval_steps_per_second': 3.71, 'epoch':                             
           7.902654867256637}                                                                                      

wandb: Adding directory to artifact (./results/google-t5/t5-small/checkpoint-450)... Done. 0.0s


[10:41:06] Evaluation metrics: {'eval_loss': 0.13813403248786926, 'eval_R1': 0.2926, 'eval_R2':     callbacks.py:39
           0.0681, 'eval_RL': 0.2805, 'eval_RLsum': 0.2803, 'eval_runtime': 7.8857,                                
           'eval_samples_per_second': 228.387, 'eval_steps_per_second': 3.678, 'epoch':                            
           8.778761061946902}                                                                                      

wandb: Adding directory to artifact (./results/google-t5/t5-small/checkpoint-500)... Done. 0.0s
wandb: Adding directory to artifact (./results/google-t5/t5-small/checkpoint-504)... Done. 0.0s


In [ ]:
import wandb
wandb.finish()

eval/R1,▁▁▁▂▄▅▇▇██
eval/R2,▁▁▁▁▂▃▄▇██
eval/RL,▁▁▁▃▄▅▆▇██
eval/RLsum,▁▁▁▂▄▅▆▇██
eval/loss,█▅▄▃▂▂▁▁▁▁
eval/runtime,█▁▁▁▁▂▂▃▂▂
eval/samples_per_second,▁███▇▇▇▆▇▇
eval/steps_per_second,▁███▇▇▇▆▇▇
train/epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/global_step,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
train/grad_norm,█▃▂▂▁▁▁▁▁▁


# Fusing Models

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig, AutoPeftModelForSeq2SeqLM
import tqdm as notebook_tqdm

In [ ]:
pwd

'/content/drive/MyDrive/GitHub/finetuning-llm'

In [ ]:
SEQUENCE_MODELS = [
        "google-t5/t5-base",
        "google-t5/t5-small"
        ]

# Load base model
# base_model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# base_model_name = "google-t5/t5-base"
base_model_name = "google-t5/t5-small"

tokenizer = AutoTokenizer.from_pretrained(
        base_model_name,
        load_in_8bit=True,
        device_map="auto",
        use_fast=True,
        padding_side="right"
        )
tokenizer.pad_token = tokenizer.eos_token
# if base_model_name in SEQUENCE_MODELS:
#   model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
# else:
#   model = AutoModelForCausalLM.from_pretrained(base_model_name)

# Load LoRA-adapted model
peft_model_local_path = "models/google-t5/t5-small/2025-02-13/02-44-10"
# peft_model_path = f"../{peft_model_local_path}"
# peft_config = PeftConfig.from_pretrained(peft_model_path)
model = AutoPeftModelForSeq2SeqLM.from_pretrained(peft_model_local_path)

# set output directory for saving models and tokenizers
output_dir = f"{peft_model_local_path}/fused_model"


In [ ]:
# fuse lora weights
model = model.merge_and_unload()

In [ ]:
tokenizer.save_pretrained(output_dir)

('models/google-t5/t5-small/2025-02-13/02-44-10/fused_model/tokenizer_config.json',
 'models/google-t5/t5-small/2025-02-13/02-44-10/fused_model/special_tokens_map.json',
 'models/google-t5/t5-small/2025-02-13/02-44-10/fused_model/spiece.model',
 'models/google-t5/t5-small/2025-02-13/02-44-10/fused_model/added_tokens.json',
 'models/google-t5/t5-small/2025-02-13/02-44-10/fused_model/tokenizer.json')

In [ ]:
model.save_pretrained(output_dir)